# 📷 OpenCV 影像輸入輸出與顯示

## 學習目標
- 掌握OpenCV的影像讀取、顯示、儲存操作
- 理解不同影像格式的特點和應用場景
- 學習影像顯示的最佳實踐
- 實作專業的影像處理工作流程

---

## 🛠️ 環境設置與導入

In [ ]:
# 導入必要的庫
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from utils.image_utils import load_image, save_image, normalize_image
from utils.visualization import display_image, display_multiple_images
from utils.performance import time_function

# 設置matplotlib顯示參數
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("🎯 OpenCV 影像I/O教學 - 環境準備完成")
print(f"OpenCV版本: {cv2.__version__}")

## 📁 1. 影像讀取 (Image Loading)

### 1.1 基本讀取方法

In [ ]:
# 1. 基本影像讀取
image_path = '../assets/images/basic/lenaColor.png'

# cv2.imread() 的讀取模式
# cv2.IMREAD_COLOR (1) - 彩色影像，預設模式
# cv2.IMREAD_GRAYSCALE (0) - 灰階影像
# cv2.IMREAD_UNCHANGED (-1) - 保持原始格式（包含透明通道）

print("📖 影像讀取模式示範：")
print("=" * 40)

# 彩色模式讀取 (預設)
img_color = cv2.imread(image_path, cv2.IMREAD_COLOR)
print(f"✅ 彩色模式: {img_color.shape if img_color is not None else 'None'}")

# 灰階模式讀取
img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
print(f"⚫ 灰階模式: {img_gray.shape if img_gray is not None else 'None'}")

# 保持原始格式
img_unchanged = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
print(f"🔄 原始格式: {img_unchanged.shape if img_unchanged is not None else 'None'}")

# 檢查檔案是否成功讀取
if img_color is not None:
    print(f"\n📐 影像資訊:")
    print(f"   尺寸: {img_color.shape[1]} x {img_color.shape[0]}")
    print(f"   通道數: {img_color.shape[2]}")
    print(f"   資料類型: {img_color.dtype}")
    print(f"   記憶體大小: {img_color.nbytes / 1024:.1f} KB")
else:
    print("❌ 影像讀取失敗！")

### 1.2 批量影像讀取

In [ ]:
# 批量讀取影像的實用函數
def load_images_from_folder(folder_path, max_images=5):
    """
    從資料夾批量載入影像
    
    Args:
        folder_path: 資料夾路徑
        max_images: 最大載入數量
    
    Returns:
        images: 影像列表
        filenames: 檔案名列表
    """
    folder = Path(folder_path)
    supported_formats = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    
    images = []
    filenames = []
    
    for i, img_path in enumerate(folder.glob('*')):
        if i >= max_images:
            break
            
        if img_path.suffix.lower() in supported_formats:
            img = cv2.imread(str(img_path))
            if img is not None:
                images.append(img)
                filenames.append(img_path.name)
                
    return images, filenames

# 示範批量載入
images, filenames = load_images_from_folder('../assets/images/basic/', max_images=3)

print(f"📁 批量載入結果: 成功載入 {len(images)} 張影像")
for i, filename in enumerate(filenames):
    print(f"   {i+1}. {filename} - {images[i].shape}")

## 🖼️ 2. 影像顯示 (Image Display)

### 2.1 OpenCV 視窗顯示（適用於桌面環境）

In [ ]:
# OpenCV 視窗顯示（主要用於桌面應用）
def show_image_opencv(image, window_name="Image", wait_key=True):
    """
    使用OpenCV顯示影像
    
    注意：此功能在Jupyter環境中可能無法正常工作
    主要用於桌面Python應用程式
    """
    if image is not None:
        # 創建可調整大小的視窗
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        cv2.imshow(window_name, image)
        
        if wait_key:
            print(f"🖼️ 顯示視窗: {window_name}")
            print("⌨️ 按任意鍵關閉視窗...")
            cv2.waitKey(0)
            cv2.destroyAllWindows()
    else:
        print("❌ 無效的影像資料")

# 因為在Jupyter環境中，我們主要使用matplotlib顯示
print("💡 提示: 在Jupyter環境中，建議使用matplotlib進行影像顯示")
print("🖥️ OpenCV的cv2.imshow()主要用於桌面應用程式")

### 2.2 Matplotlib 顯示（推薦用於Jupyter）

In [ ]:
# Matplotlib 顯示 - Jupyter 環境推薦方法
if img_color is not None:
    # 顯示彩色和灰階影像對比
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 彩色影像 (BGR -> RGB轉換)
    img_rgb = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)
    axes[0].imshow(img_rgb)
    axes[0].set_title('🎨 彩色影像 (RGB)')
    axes[0].axis('off')
    
    # 灰階影像
    axes[1].imshow(img_gray, cmap='gray')
    axes[1].set_title('⚫ 灰階影像')
    axes[1].axis('off')
    
    # 通道分離顯示
    b, g, r = cv2.split(img_color)
    merged_channels = np.hstack([r, g, b])
    axes[2].imshow(merged_channels, cmap='gray')
    axes[2].set_title('📊 RGB通道分離 (R|G|B)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 顯示影像統計資訊
    print("📊 影像統計資訊:")
    print(f"   平均像素值: {img_color.mean():.2f}")
    print(f"   標準差: {img_color.std():.2f}")
    print(f"   最小值: {img_color.min()}")
    print(f"   最大值: {img_color.max()}")

### 2.3 進階顯示功能

In [ ]:
# 進階顯示功能：影像資訊疊加
def display_image_with_info(image, title="Image"):
    """
    顯示影像並疊加詳細資訊
    """
    if image is None:
        print("❌ 無效的影像")
        return
    
    # 創建顯示用的影像副本
    display_img = image.copy()
    
    # 轉換為RGB（如果是彩色影像）
    if len(image.shape) == 3:
        display_img = cv2.cvtColor(display_img, cv2.COLOR_BGR2RGB)
        
    # 創建圖形
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 顯示影像
    if len(image.shape) == 3:
        ax1.imshow(display_img)
    else:
        ax1.imshow(display_img, cmap='gray')
    ax1.set_title(f'🖼️ {title}')
    ax1.axis('off')
    
    # 顯示直方圖
    if len(image.shape) == 3:
        colors = ['red', 'green', 'blue']
        for i, color in enumerate(colors):
            hist = cv2.calcHist([image], [i], None, [256], [0, 256])
            ax2.plot(hist, color=color, label=f'{color.upper()} channel')
        ax2.legend()
    else:
        hist = cv2.calcHist([image], [0], None, [256], [0, 256])
        ax2.plot(hist, color='black')
    
    ax2.set_title('📈 像素值分布直方圖')
    ax2.set_xlabel('像素值')
    ax2.set_ylabel('頻率')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 打印詳細資訊
    print(f"📋 {title} 詳細資訊:")
    print(f"   🏷️ 尺寸: {image.shape}")
    print(f"   💾 資料類型: {image.dtype}")
    print(f"   📊 記憶體使用: {image.nbytes / 1024:.1f} KB")
    
    if len(image.shape) == 3:
        for i, channel in enumerate(['藍', '綠', '紅']):
            channel_data = image[:, :, i]
            print(f"   {channel}色通道: 平均={channel_data.mean():.1f}, 標準差={channel_data.std():.1f}")
    else:
        print(f"   📊 統計: 平均={image.mean():.1f}, 標準差={image.std():.1f}")

# 示範進階顯示
if img_color is not None:
    display_image_with_info(img_color, "Lena彩色影像")

## 💾 3. 影像儲存 (Image Saving)

### 3.1 基本儲存操作

In [ ]:
# 創建輸出目錄
output_dir = Path('../assets/images/output')
output_dir.mkdir(exist_ok=True)

# 基本儲存操作
if img_color is not None:
    print("💾 影像儲存示範:")
    print("=" * 30)
    
    # 1. 儲存為不同格式
    formats = {
        'jpg': '高壓縮比，適合照片',
        'png': '無損壓縮，支援透明度',
        'bmp': '無壓縮，檔案較大',
        'tiff': '專業格式，支援多層'
    }
    
    for fmt, description in formats.items():
        output_path = output_dir / f'test_image.{fmt}'
        success = cv2.imwrite(str(output_path), img_color)
        
        if success:
            file_size = output_path.stat().st_size / 1024
            print(f"✅ {fmt.upper()}: {file_size:.1f} KB - {description}")
        else:
            print(f"❌ {fmt.upper()}: 儲存失敗")
    
    # 2. 帶品質參數的JPEG儲存
    quality_levels = [30, 50, 80, 95]
    print("\n🎚️ JPEG品質比較:")
    
    for quality in quality_levels:
        output_path = output_dir / f'test_quality_{quality}.jpg'
        success = cv2.imwrite(str(output_path), img_color, 
                             [cv2.IMWRITE_JPEG_QUALITY, quality])
        
        if success:
            file_size = output_path.stat().st_size / 1024
            print(f"   品質{quality}%: {file_size:.1f} KB")
    
    # 3. PNG壓縮等級
    compression_levels = [0, 5, 9]  # 0=無壓縮, 9=最大壓縮
    print("\n🗜️ PNG壓縮等級比較:")
    
    for level in compression_levels:
        output_path = output_dir / f'test_compression_{level}.png'
        success = cv2.imwrite(str(output_path), img_color,
                             [cv2.IMWRITE_PNG_COMPRESSION, level])
        
        if success:
            file_size = output_path.stat().st_size / 1024
            print(f"   壓縮等級{level}: {file_size:.1f} KB")

print(f"\n📁 所有檔案已儲存至: {output_dir}")

### 3.2 批量儲存與檔案管理

In [ ]:
# 批量處理和儲存
def batch_save_images(images, filenames, output_folder, format='jpg', quality=85):
    """
    批量儲存影像
    
    Args:
        images: 影像列表
        filenames: 檔案名列表
        output_folder: 輸出資料夾
        format: 輸出格式
        quality: JPEG品質（僅當format='jpg'時有效）
    """
    output_path = Path(output_folder)
    output_path.mkdir(exist_ok=True)
    
    saved_count = 0
    total_size = 0
    
    for img, filename in zip(images, filenames):
        # 生成新的檔案名
        base_name = Path(filename).stem
        new_filename = f"{base_name}_processed.{format}"
        output_file = output_path / new_filename
        
        # 設定儲存參數
        if format.lower() == 'jpg':
            params = [cv2.IMWRITE_JPEG_QUALITY, quality]
        elif format.lower() == 'png':
            params = [cv2.IMWRITE_PNG_COMPRESSION, 6]
        else:
            params = []
        
        # 儲存影像
        success = cv2.imwrite(str(output_file), img, params)
        
        if success:
            file_size = output_file.stat().st_size
            total_size += file_size
            saved_count += 1
            print(f"✅ {new_filename}: {file_size/1024:.1f} KB")
        else:
            print(f"❌ {new_filename}: 儲存失敗")
    
    print(f"\n📊 批量儲存結果:")
    print(f"   成功儲存: {saved_count}/{len(images)} 個檔案")
    print(f"   總大小: {total_size/1024:.1f} KB")
    print(f"   平均大小: {total_size/saved_count/1024:.1f} KB" if saved_count > 0 else "")

# 示範批量儲存
if images:
    batch_save_images(images, filenames, output_dir / 'batch_output', 'jpg', 80)

## 📊 4. 檔案格式深入分析

### 4.1 格式特性比較

In [ ]:
# 檔案格式特性分析
def analyze_image_formats(image):
    """
    分析不同格式的檔案大小和載入時間
    """
    if image is None:
        return
    
    formats_config = {
        'bmp': ([], '無壓縮位圖'),
        'png': ([cv2.IMWRITE_PNG_COMPRESSION, 6], '無損壓縮'),
        'jpg_low': ([cv2.IMWRITE_JPEG_QUALITY, 30], 'JPEG低品質'),
        'jpg_mid': ([cv2.IMWRITE_JPEG_QUALITY, 70], 'JPEG中品質'),
        'jpg_high': ([cv2.IMWRITE_JPEG_QUALITY, 95], 'JPEG高品質'),
    }
    
    results = []
    
    for fmt_name, (params, description) in formats_config.items():
        # 確定副檔名
        ext = 'jpg' if 'jpg' in fmt_name else fmt_name
        temp_file = output_dir / f'test_format.{ext}'
        
        # 測量儲存時間
        import time
        start_time = time.time()
        success = cv2.imwrite(str(temp_file), image, params)
        save_time = (time.time() - start_time) * 1000  # 轉為毫秒
        
        if success:
            # 獲取檔案大小
            file_size = temp_file.stat().st_size
            
            # 測量載入時間
            start_time = time.time()
            loaded_img = cv2.imread(str(temp_file))
            load_time = (time.time() - start_time) * 1000  # 轉為毫秒
            
            # 計算壓縮比
            original_size = image.nbytes
            compression_ratio = original_size / file_size
            
            results.append({
                'format': fmt_name,
                'description': description,
                'file_size': file_size,
                'save_time': save_time,
                'load_time': load_time,
                'compression_ratio': compression_ratio
            })
            
            # 清理臨時檔案
            temp_file.unlink()
    
    # 顯示結果表格
    print("📊 檔案格式性能分析:")
    print("=" * 80)
    print(f"{'格式':<10} {'描述':<12} {'大小(KB)':<10} {'壓縮比':<8} {'存檔(ms)':<10} {'載入(ms)':<10}")
    print("-" * 80)
    
    for result in results:
        print(f"{result['format']:<10} {result['description']:<12} "
              f"{result['file_size']/1024:<10.1f} {result['compression_ratio']:<8.1f} "
              f"{result['save_time']:<10.2f} {result['load_time']:<10.2f}")
    
    return results

# 執行格式分析
if img_color is not None:
    format_results = analyze_image_formats(img_color)

### 4.2 視覺化格式比較

In [ ]:
# 視覺化格式比較
if 'format_results' in locals() and format_results:
    # 創建比較圖表
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    formats = [r['format'] for r in format_results]
    file_sizes = [r['file_size']/1024 for r in format_results]
    compression_ratios = [r['compression_ratio'] for r in format_results]
    save_times = [r['save_time'] for r in format_results]
    load_times = [r['load_time'] for r in format_results]
    
    # 檔案大小比較
    bars1 = ax1.bar(formats, file_sizes, color='skyblue')
    ax1.set_title('📦 檔案大小比較 (KB)')
    ax1.set_ylabel('大小 (KB)')
    ax1.tick_params(axis='x', rotation=45)
    
    # 在柱狀圖上顯示數值
    for bar, size in zip(bars1, file_sizes):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{size:.1f}', ha='center', va='bottom')
    
    # 壓縮比比較
    bars2 = ax2.bar(formats, compression_ratios, color='lightgreen')
    ax2.set_title('🗜️ 壓縮比比較')
    ax2.set_ylabel('壓縮比')
    ax2.tick_params(axis='x', rotation=45)
    
    for bar, ratio in zip(bars2, compression_ratios):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f'{ratio:.1f}x', ha='center', va='bottom')
    
    # 儲存時間比較
    bars3 = ax3.bar(formats, save_times, color='orange')
    ax3.set_title('💾 儲存時間比較 (ms)')
    ax3.set_ylabel('時間 (ms)')
    ax3.tick_params(axis='x', rotation=45)
    
    # 載入時間比較
    bars4 = ax4.bar(formats, load_times, color='pink')
    ax4.set_title('📖 載入時間比較 (ms)')
    ax4.set_ylabel('時間 (ms)')
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## ⚡ 5. 性能優化與最佳實踐

### 5.1 記憶體效率優化

In [ ]:
# 記憶體效率的影像處理
import psutil
import gc

def memory_efficient_processing():
    """
    示範記憶體效率的影像處理方法
    """
    print("🧠 記憶體效率優化示範:")
    print("=" * 40)
    
    # 獲取初始記憶體使用量
    process = psutil.Process()
    initial_memory = process.memory_info().rss / 1024 / 1024  # MB
    print(f"初始記憶體使用: {initial_memory:.1f} MB")
    
    # 載入多個大影像（模擬）
    large_images = []
    for i in range(5):
        # 創建大型隨機影像
        large_img = np.random.randint(0, 255, (1000, 1000, 3), dtype=np.uint8)
        large_images.append(large_img)
    
    current_memory = process.memory_info().rss / 1024 / 1024
    print(f"載入5個大影像後: {current_memory:.1f} MB (+{current_memory-initial_memory:.1f} MB)")
    
    # 處理完後立即釋放記憶體
    del large_images
    gc.collect()  # 強制垃圾回收
    
    final_memory = process.memory_info().rss / 1024 / 1024
    print(f"釋放記憶體後: {final_memory:.1f} MB")
    
    # 記憶體使用建議
    print("\n💡 記憶體優化建議:")
    print("   1. 使用完影像後立即del變數")
    print("   2. 定期調用gc.collect()")
    print("   3. 避免不必要的影像複製")
    print("   4. 使用適當的資料類型（uint8 vs float64）")
    print("   5. 考慮影像分塊處理大檔案")

memory_efficient_processing()

### 5.2 批量處理性能測試

In [ ]:
# 批量處理性能測試
@time_function
def batch_resize_images(images, target_size=(512, 512)):
    """
    批量調整影像大小
    """
    resized_images = []
    for img in images:
        resized = cv2.resize(img, target_size)
        resized_images.append(resized)
    return resized_images

@time_function
def batch_convert_to_grayscale(images):
    """
    批量轉換為灰階
    """
    gray_images = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_images.append(gray)
    return gray_images

# 性能測試
if images:
    print("⚡ 批量處理性能測試:")
    print("=" * 30)
    
    # 測試調整大小
    resized_images = batch_resize_images(images)
    print(f"✅ 完成 {len(images)} 張影像調整大小")
    
    # 測試灰階轉換
    gray_images = batch_convert_to_grayscale(images)
    print(f"✅ 完成 {len(images)} 張影像灰階轉換")
    
    # 顯示結果
    if len(resized_images) > 0:
        fig, axes = plt.subplots(1, min(3, len(resized_images)), figsize=(12, 4))
        if len(resized_images) == 1:
            axes = [axes]
        
        for i, (ax, img) in enumerate(zip(axes, resized_images[:3])):
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            ax.set_title(f'調整大小後 #{i+1}')
            ax.axis('off')
        
        plt.tight_layout()
        plt.show()

## 🎯 6. 實戰應用：影像處理工作流程

### 6.1 完整的影像處理管道

In [ ]:
# 完整的影像處理工作流程
class ImageProcessor:
    """
    專業的影像處理工作流程類
    """
    
    def __init__(self, input_folder, output_folder):
        self.input_folder = Path(input_folder)
        self.output_folder = Path(output_folder)
        self.output_folder.mkdir(exist_ok=True)
        self.processed_count = 0
        self.failed_count = 0
        
    def load_images(self, max_images=None):
        """
        載入資料夾中的所有影像
        """
        supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
        image_files = []
        
        for file_path in self.input_folder.iterdir():
            if file_path.suffix.lower() in supported_formats:
                image_files.append(file_path)
                if max_images and len(image_files) >= max_images:
                    break
        
        return image_files
    
    def process_single_image(self, image_path, operations=None):
        """
        處理單一影像
        
        Args:
            image_path: 影像路徑
            operations: 處理操作列表
        """
        try:
            # 載入影像
            image = cv2.imread(str(image_path))
            if image is None:
                raise ValueError(f"無法載入影像: {image_path}")
            
            processed_image = image.copy()
            
            # 執行處理操作
            if operations:
                for operation in operations:
                    processed_image = operation(processed_image)
            
            # 儲存處理後的影像
            output_path = self.output_folder / f"processed_{image_path.name}"
            success = cv2.imwrite(str(output_path), processed_image)
            
            if success:
                self.processed_count += 1
                return True, output_path
            else:
                raise ValueError("儲存失敗")
                
        except Exception as e:
            self.failed_count += 1
            return False, str(e)
    
    def batch_process(self, operations=None, max_images=5):
        """
        批量處理影像
        """
        image_files = self.load_images(max_images)
        results = []
        
        print(f"🚀 開始批量處理 {len(image_files)} 張影像...")
        print("=" * 50)
        
        for i, image_path in enumerate(image_files, 1):
            print(f"處理進度: {i}/{len(image_files)} - {image_path.name}")
            
            success, result = self.process_single_image(image_path, operations)
            results.append((image_path.name, success, result))
            
            if success:
                print(f"  ✅ 成功: {result}")
            else:
                print(f"  ❌ 失敗: {result}")
        
        print("\n📊 處理完成統計:")
        print(f"   成功: {self.processed_count} 張")
        print(f"   失敗: {self.failed_count} 張")
        print(f"   輸出目錄: {self.output_folder}")
        
        return results

# 定義一些基本的影像處理操作
def resize_operation(image, size=(512, 512)):
    """調整影像大小"""
    return cv2.resize(image, size)

def brightness_operation(image, value=30):
    """調整亮度"""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 2] = cv2.add(hsv[:, :, 2], value)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def blur_operation(image, kernel_size=5):
    """高斯模糊"""
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

# 建立處理器並執行批量處理
processor = ImageProcessor('../assets/images/basic', output_dir / 'workflow_output')

# 定義處理流程
operations = [
    resize_operation,
    lambda img: brightness_operation(img, 20),
    lambda img: blur_operation(img, 3)
]

# 執行批量處理
results = processor.batch_process(operations, max_images=3)

## 📝 7. 課程總結與最佳實踐

### ✅ 已學習的核心技能：

1. **影像輸入操作**
   - `cv2.imread()` 的各種讀取模式
   - 批量影像載入技術
   - 錯誤處理和驗證

2. **影像顯示技術**
   - OpenCV視窗顯示（桌面應用）
   - Matplotlib顯示（Jupyter環境）
   - 進階視覺化和資訊疊加

3. **影像儲存優化**
   - 多種格式的特性和應用
   - 壓縮參數調整
   - 批量處理和檔案管理

4. **性能優化**
   - 記憶體效率管理
   - 批量處理性能測試
   - 專業工作流程設計

### 🏆 最佳實踐建議：

1. **檔案格式選擇**
   - 照片：JPEG (品質70-85%)
   - 螢幕截圖/圖表：PNG
   - 專業處理：TIFF
   - 快速原型：BMP（開發階段）

2. **記憶體管理**
   - 及時釋放大型影像變數
   - 使用適當的資料類型
   - 考慮分塊處理大檔案

3. **錯誤處理**
   - 始終檢查影像是否成功載入
   - 驗證檔案路徑和格式
   - 提供有意義的錯誤訊息

4. **工作流程設計**
   - 建立可重用的處理函數
   - 實現批量處理能力
   - 記錄處理過程和結果

### 🚀 下一步學習方向：

1. **基本變換** (`basic_transformations.ipynb`)
   - 幾何變換（旋轉、縮放、翻轉）
   - 仿射變換和透視變換

2. **色彩空間** (`color_spaces.ipynb`)
   - HSV、LAB等色彩空間
   - 色彩分析和處理

3. **影像前處理** (`03_preprocessing/`)
   - 濾波和去噪
   - 形態學操作
   - 邊緣檢測

---
*🎯 OpenCV影像I/O教學完成 - 您已掌握專業的影像處理基礎技能！*